### W23P1 STAT 857 - Modeling

In [ ]:
pip install xgboost lightgbm catboost

In [10]:
## Importing libraries
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor 
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', 100)

In [11]:
## Reading the data
train = pd.read_csv('Data/W23P1_train_final.csv')
test = pd.read_csv('Data/W23P1_test_final.csv')
sub = pd.read_csv('Data/W23P1_sample_submission.csv')

### Baseline Models:

In [ ]:
## Defining input and target variables
X_train = train.drop(columns = ['fare_amount'])
Y_train = train['fare_amount']

X_test = test

In [ ]:
## Random Forest Model:
rf_md = RandomForestRegressor(max_depth = 3, n_estimators = 500).fit(X_train, Y_train)

sub['fare_amount'] = rf_md.predict(X_test)

sub.to_csv('Submissions/rf_baseline.csv', index = False)

In [ ]:
## XGBoost Model:
XGB_md = XGBRegressor(tree_method = 'hist', colsample_bytree = 0.7, gamma = 0.8, learning_rate = 0.01, max_depth = 7, 
                      min_child_weight = 10, n_estimators = 1000, subsample = 0.7).fit(X_train, Y_train)

sub['fare_amount'] = XGB_md.predict(X_test)

sub.to_csv('Submissions/xgb_baseline.csv', index = False)

In [ ]:
## LightGBM Model:
lgb_md = LGBMRegressor(n_estimators = 1000, max_depth = 7, learning_rate = 0.01, num_leaves = 20, lambda_l1 = 3, lambda_l2 = 3, 
                       bagging_fraction = 0.7, feature_fraction = 0.7).fit(X_train, Y_train)

sub['fare_amount'] = lgb_md.predict(X_test)

sub.to_csv('Submissions/lgbm_baseline.csv', index = False)

In [ ]:
## CatBoost Model:
cat_md = CatBoostRegressor(loss_function = 'RMSE', iterations = 1000, learning_rate = 0.01, depth = 7, random_strength = 0.5, 
                           bagging_temperature = 0.7, border_count = 30, l2_leaf_reg = 5, verbose = False).fit(X_train, Y_train)

sub['fare_amount'] = cat_md.predict(X_test)

sub.to_csv('Submissions/cat_baseline.csv', index = False)

### Second Round of Models: with optimized parameters

In [ ]:
## Random Forest Model:
rf_md = RandomForestRegressor(max_depth = 300, n_estimators = 12, min_samples_split = 5, 
                              min_samples_leaf = 6).fit(X_train, Y_train)

rf_preds = rf_md.predict(X_test)

sub['fare_amount'] = rf_preds

sub.to_csv('Submissions/rf_rd2.csv', index = False)

In [ ]:
## XGBoost Model:
XGB_md = XGBRegressor(tree_method = 'hist', n_estimators = 500, learning_rate = 0.02, max_depth = 5, gamma = 0.2, 
                      min_child_weight = 10, subsample = 0.94, colsample_bytree = 0.92, verbosity = 0).fit(X_train, Y_train)

XGB_preds = XGB_md.predict(X_test)

sub['fare_amount'] = XGB_preds

sub.to_csv('Submissions/xgb_rd2.csv', index = False)

In [ ]:
## LightGBM Model:
lgbm_md = LGBMRegressor(boosting_type = 'dart', n_estimators = 600, learning_rate = 0.18, num_leaves = 8, max_depth = 8,
                      subsample = 0.73, colsample_bytree = 0.86, random_state = 543, reg_alpha = 0.021, reg_lambda = 0.027, 
                        objective = 'rmse', verbosity = -1).fit(X_train, Y_train)

lgbm_preds = lgbm_md.predict(X_test)

sub['fare_amount'] = lgbm_preds

sub.to_csv('Submissions/lgbm_rd2.csv', index = False)

In [ ]:
## Ensemble Model:

## Constructing the training data
rf_preds_train = rf_md.predict(X_train)
XGB_preds_train = XGB_md.predict(X_train)
lgbm_preds_train = lgbm_md.predict(X_train)

X_train_ensemble = pd.DataFrame({'rf': rf_preds_train, 'xgb': XGB_preds_train, 'lgbm': lgbm_preds_train})
X_test_ensemble = pd.DataFrame({'rf': rf_preds, 'xgb': XGB_preds, 'lgbm': lgbm_preds})

## Building the model
ensemble_md = RandomForestRegressor(max_depth = 3, n_estimators = 500).fit(X_train_ensemble, Y_train)

sub['fare_amount'] = ensemble_md.predict(X_test_ensemble)

sub.to_csv('Submissions/ensemble_rd2.csv', index = False)

### Third Round of Models:

In [12]:
## Defining input and target variables

variables = ['distance', 'haversine', 'duration', 'passenger_count', 'pickup_day','Monday', 'Tuesday', 'Wednesday', 'Thursday',  
             'Friday', 'Saturday','weekend', 'pickup_hour', 'rush_hour', 'overnight', 'pickup_LGA', 'dropoff_LGA', 'pickup_JFK', 
             'dropoff_JFK', 'pickup_EWR', 'dropoff_EWR', 'airport', 'change_borough', 'pickup_longitude', 'pickup_latitude', 
             'dropoff_longitude', 'dropoff_latitude']

X_train = train[variables]
Y_train = train['fare_amount']

X_test = test[variables]

In [18]:
## Random Forest Model:
rf_md = RandomForestRegressor(max_depth = 300, n_estimators = 12, min_samples_split = 5, 
                              min_samples_leaf = 6).fit(X_train, Y_train)

rf_preds = rf_md.predict(X_test)

sub['fare_amount'] = rf_preds

#sub.to_csv('Submissions/rf_rd3.csv', index = False)

In [19]:
## XGBoost Model:
XGB_md = XGBRegressor(tree_method = 'hist', n_estimators = 500, learning_rate = 0.02, max_depth = 5, gamma = 0.2, 
                      min_child_weight = 10, subsample = 0.94, colsample_bytree = 0.92, verbosity = 0).fit(X_train, Y_train)

XGB_preds = XGB_md.predict(X_test)

sub['fare_amount'] = XGB_preds

#sub.to_csv('Submissions/xgb_rd3.csv', index = False)

In [20]:
## LightGBM Model:
lgbm_md = LGBMRegressor(boosting_type = 'dart', n_estimators = 600, learning_rate = 0.18, num_leaves = 8, max_depth = 8,
                      subsample = 0.73, colsample_bytree = 0.86, random_state = 543, reg_alpha = 0.021, reg_lambda = 0.027, 
                        objective = 'rmse', verbosity = -1).fit(X_train, Y_train)

lgbm_preds = lgbm_md.predict(X_test)

sub['fare_amount'] = lgbm_preds

#sub.to_csv('Submissions/lgbm_rd3.csv', index = False)

In [7]:
## Ensemble Model:

## Constructing the training data
rf_preds_train = rf_md.predict(X_train)
XGB_preds_train = XGB_md.predict(X_train)
lgbm_preds_train = lgbm_md.predict(X_train)

X_train_ensemble = pd.DataFrame({'rf': rf_preds_train, 'xgb': XGB_preds_train, 'lgbm': lgbm_preds_train})
X_test_ensemble = pd.DataFrame({'rf': rf_preds, 'xgb': XGB_preds, 'lgbm': lgbm_preds})

## Building the model
ensemble_md = RandomForestRegressor(max_depth = 3, n_estimators = 500).fit(X_train_ensemble, Y_train)

sub['fare_amount'] = ensemble_md.predict(X_test_ensemble)

sub.to_csv('Submissions/ensemble_rd3.csv', index = False)

In [27]:
ins = test.merge(train, how = 'inner', on = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'])
ins.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,distance_x,haversine_x,duration_x,passenger_count_x,pickup_day_x,holiday_x,Monday_x,Tuesday_x,Wednesday_x,Thursday_x,Friday_x,Saturday_x,weekend_x,pickup_hour_x,rush_hour_x,overnight_x,airport_x,LGA_x,JFK_x,EWR_x,pickup_LGA_x,dropoff_LGA_x,pickup_JFK_x,dropoff_JFK_x,pickup_EWR_x,dropoff_EWR_x,pickup_airport_x,dropoff_airport_x,change_borough_x,pickup_bronx_x,pickup_brooklyn_x,pickup_manhattan_x,pickup_queens_x,pickup_staten_island_x,dropoff_bronx_x,dropoff_brooklyn_x,dropoff_manhattan_x,dropoff_queens_x,dropoff_staten_island_x,fare_amount,distance_y,haversine_y,duration_y,passenger_count_y,pickup_day_y,holiday_y,Monday_y,Tuesday_y,Wednesday_y,Thursday_y,Friday_y,Saturday_y,weekend_y,pickup_hour_y,rush_hour_y,overnight_y,airport_y,LGA_y,JFK_y,EWR_y,pickup_LGA_y,dropoff_LGA_y,pickup_JFK_y,dropoff_JFK_y,pickup_EWR_y,dropoff_EWR_y,pickup_airport_y,dropoff_airport_y,change_borough_y,pickup_bronx_y,pickup_brooklyn_y,pickup_manhattan_y,pickup_queens_y,pickup_staten_island_y,dropoff_bronx_y,dropoff_brooklyn_y,dropoff_manhattan_y,dropoff_queens_y,dropoff_staten_island_y
0,-73.945297,40.783367,-73.945297,40.783367,1.0,0.0,1,1,15,0,0,1,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,6.5,1.0,0.0,1,1,12,0,0,0,0,0,0,1,1,22,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
1,-73.945297,40.783367,-73.945297,40.783367,1.0,0.0,1,1,15,0,0,1,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,6.5,1.0,0.0,1,1,8,0,0,1,0,0,0,0,0,21,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
2,-73.945297,40.783367,-73.945297,40.783367,1.0,0.0,1,1,8,0,0,1,0,0,0,0,0,22,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,6.5,1.0,0.0,1,1,12,0,0,0,0,0,0,1,1,22,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
3,-73.945297,40.783367,-73.945297,40.783367,1.0,0.0,1,1,8,0,0,1,0,0,0,0,0,22,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,6.5,1.0,0.0,1,1,8,0,0,1,0,0,0,0,0,21,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
4,-73.945297,40.783367,-73.945297,40.783367,1.0,0.0,1,1,11,0,0,0,0,0,1,0,0,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,6.5,1.0,0.0,1,1,12,0,0,0,0,0,0,1,1,22,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0


In [28]:
sub['fare_amount'] = (rf_preds + XGB_preds + lgbm_preds) / 3
print(np.mean(sub['fare_amount']))

ind = [631,  1044,  8718,  9614,  9784, 11654, 12032, 12810, 13254, 13374, 16042, 17660, 18328, 18930, 20616, 22499, 22829, 23702, \
       25597, 25958, 26925, 27347, 27552, 31726, 33099, 33668]
fare = [6.5,  6.5,  6.66666667, 6.5, 6.5, 11.4, 14.25, 14.25, 6.5, 5.5, 51.625, 51.625, 9.5, 5.5, 11.4, 18.75, 13.5, 6.5, 18.75, 14.25, \
        6.66666667, 7.75, 6.5, 14.25, 6.5, 14.25]    

sub['fare_amount'].iloc[ind] = fare

print(np.mean(sub['fare_amount']))
sub.to_csv('Submissions/ensemble_average4.csv', index = False)

# sub['fare_amount'] = (rf_preds + 3*XGB_preds + 2*lgbm_preds) / 6

# sub.to_csv('Submissions/ensemble_average3.csv', index = False)

11.81092269241668
11.813191934411623


/tmp/ipykernel_14427/4112006895.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub['fare_amount'].iloc[ind] = fare


### Fourth Round: Optimized HPs and top-10 variables

In [4]:
## Defining input and target variables

variables = ['distance', 'haversine', 'dropoff_longitude', 'duration', 'pickup_longitude', 'dropoff_EWR', 'EWR', 
             'dropoff_airport', 'pickup_airport', 'dropoff_JFK']

X_train = train[variables]
Y_train = train['fare_amount']

X_test = test[variables]

In [5]:
## Random Forest Model:
rf_md = RandomForestRegressor(max_depth = 100, n_estimators = 10, min_samples_split = 8, 
                              min_samples_leaf = 8).fit(X_train, Y_train)

rf_preds = rf_md.predict(X_test)

sub['fare_amount'] = rf_preds

sub.to_csv('Submissions/rf_rd4.csv', index = False)

In [6]:
## XGBoost Model:
XGB_md = XGBRegressor(tree_method = 'hist', n_estimators = 500, learning_rate = 0.01, max_depth = 10, gamma = 0.13, 
                      min_child_weight = 15, subsample = 0.71, colsample_bytree = 0.7, verbosity = 0).fit(X_train, Y_train)

XGB_preds = XGB_md.predict(X_test)

sub['fare_amount'] = XGB_preds

sub.to_csv('Submissions/xgb_rd4.csv', index = False)

In [7]:
## LightGBM Model:
lgbm_md = LGBMRegressor(boosting_type = 'dart', n_estimators = 500, learning_rate = 0.1, num_leaves = 29, max_depth = 3,
                      subsample = 0.93, colsample_bytree = 0.73, random_state = 433, reg_alpha = 0.065, reg_lambda = 0.047, 
                        objective = 'rmse', verbosity = -1).fit(X_train, Y_train)

lgbm_preds = lgbm_md.predict(X_test)

sub['fare_amount'] = lgbm_preds

sub.to_csv('Submissions/lgbm_rd4.csv', index = False)

In [8]:
## Ensemble Model:

## Constructing the training data
rf_preds_train = rf_md.predict(X_train)
XGB_preds_train = XGB_md.predict(X_train)
lgbm_preds_train = lgbm_md.predict(X_train)

X_train_ensemble = pd.DataFrame({'rf': rf_preds_train, 'xgb': XGB_preds_train, 'lgbm': lgbm_preds_train})
X_test_ensemble = pd.DataFrame({'rf': rf_preds, 'xgb': XGB_preds, 'lgbm': lgbm_preds})

## Building the model
ensemble_md = RandomForestRegressor(max_depth = 5, n_estimators = 1000).fit(X_train_ensemble, Y_train)

sub['fare_amount'] = ensemble_md.predict(X_test_ensemble)

sub.to_csv('Submissions/ensemble_rd4.csv', index = False)

### Fifth Round of Models: Optimized hps, new features, with top __ features 

In [ ]:
## Defining input and target variables
X_train_XGB = train.drop(columns = ['fare_amount'])
Y_train_XGB = train['fare_amount']

X_test = test

## XGBoost Model:
XGB_md = XGBRegressor(tree_method = 'hist', n_estimators = 300, learning_rate = 0.02, max_depth = 5, gamma = 0.2, 
                      min_child_weight = 6, subsample = 0.99, colsample_bytree = 0.74, verbosity = 0).fit(X_train_XGB, Y_train_XGB)

XGB_preds = XGB_md.predict(X_test)

sub['fare_amount'] = XGB_preds

sub.to_csv('Submissions/xgb_rd3a.csv', index = False)

In [ ]:
## Defining input and target variables
X_train_XGB = train.drop(columns = ['fare_amount', 'Tuesday', 'Wednesday', 'dropoff_LGA', 'pickup_JFK', 'pickup_EWR', 'dropoff_EWR', 
                                   'pickup_bronx', 'pickup_brooklyn', 'pickup_staten_island', 'dropoff_bronx', 'dropoff_brooklyn', 
                                   'dropoff_queens', 'dropoff_staten_island'])
Y_train_XGB = train['fare_amount']

X_test = test.drop(columns = ['Tuesday', 'Wednesday', 'dropoff_LGA', 'pickup_JFK', 'pickup_EWR', 'dropoff_EWR', 'pickup_bronx', 
                              'pickup_brooklyn', 'pickup_staten_island', 'dropoff_bronx', 'dropoff_brooklyn', 'dropoff_queens', 
                              'dropoff_staten_island'])

## XGBoost Model:
XGB_md = XGBRegressor(tree_method = 'hist', n_estimators = 300, learning_rate = 0.02, max_depth = 5, gamma = 0.2, 
                      min_child_weight = 6, subsample = 0.99, colsample_bytree = 0.74, verbosity = 0).fit(X_train_XGB, Y_train_XGB)

XGB_preds = XGB_md.predict(X_test)

sub['fare_amount'] = XGB_preds

sub.to_csv('Submissions/xgb_rd3b.csv', index = False)

In [ ]:
## Defining input and target variables
X_train_lgbm = train.drop(columns = ['fare_amount'])
Y_train_lgbm = train['fare_amount']

X_test = test

## LightGBM Model:
lgbm_md = LGBMRegressor(boosting_type = 'dart', n_estimators = 1200, learning_rate = 0.06, num_leaves = 9, max_depth = 3,
                      subsample = 0.82, colsample_bytree = 0.9, random_state = 660, reg_alpha = 0.042, reg_lambda = 0.066, 
                        objective = 'rmse', verbosity = -1).fit(X_train_lgbm, Y_train_lgbm)

lgbm_preds = lgbm_md.predict(X_test)

sub['fare_amount'] = lgbm_preds

sub.to_csv('Submissions/lgbm_rd3a.csv', index = False)

In [ ]:
## Defining input and target variables
X_train_lgbm = train.drop(columns = ['fare_amount', 'Tuesday', 'Wednesday', 'dropoff_LGA', 'pickup_JFK', 'pickup_EWR', 'dropoff_EWR', 
                                    'pickup_bronx', 'pickup_brooklyn', 'pickup_staten_island', 'dropoff_bronx', 'dropoff_brooklyn', 
                                    'dropoff_queens', 'dropoff_staten_island'])
Y_train_lgbm = train['fare_amount']

X_test = test.drop(columns = ['Tuesday', 'Wednesday', 'dropoff_LGA', 'pickup_JFK', 'pickup_EWR', 'dropoff_EWR', 'pickup_bronx', 
                              'pickup_brooklyn', 'pickup_staten_island', 'dropoff_bronx', 'dropoff_brooklyn', 'dropoff_queens', 
                              'dropoff_staten_island'])

## LightGBM Model:
lgbm_md = LGBMRegressor(boosting_type = 'dart', n_estimators = 1200, learning_rate = 0.06, num_leaves = 9, max_depth = 3,
                      subsample = 0.82, colsample_bytree = 0.9, random_state = 660, reg_alpha = 0.042, reg_lambda = 0.066, 
                        objective = 'rmse', verbosity = -1).fit(X_train_lgbm, Y_train_lgbm)

lgbm_preds = lgbm_md.predict(X_test)

sub['fare_amount'] = lgbm_preds

sub.to_csv('Submissions/lgbm_rd3b.csv', index = False)